In [15]:
import pandas as pd
from IPython.core.display import clear_output
import numpy as np
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError

import sys
sys.path.append('./modules/')
from functions import retry

In [16]:
def create_driver():
  # WINDOWS
  # chromedriver_path = str("""C:/Users/Brian/Desktop/chromedriver.exe""")
  # user_data_path = "C:/Users/Brian/AppData/Local/Google/Chrome/User Data/Thread_Five"

  # MAC
  chromedriver_path = "/Users/brianphelps/Desktop/chromedriver"
  user_data_path = "/Users/brianphelps/Library/Application Support/Google/Chrome/Thread_Four"

  # OPTIONS
  options = webdriver.ChromeOptions()
  options.add_argument("no-sandbox")
  options.add_argument("disable-dev-shm-usage")
  options.add_argument("headless")
  options.add_argument(f"user-data-dir={user_data_path}")
  
  # START UP WEB DRIVER
  driver = webdriver.Chrome(chromedriver_path, chrome_options=options)
  return driver

In [ ]:
class GetHousingData(object):
  def __init__(self,state,city=''):
    """
      Zillow Scraper
      
      Params
      ======
      
      state (str) : Target state for acquiring housing data.
                  (e.g., Texas)
                  
      city (str) : Target city for acquiring housing data.
                  (e.g., Austin)
                  
      
    """
    super().__init__()
    self.city=city
    self.state=state
    
    self.houses = pd.DataFrame()

  def _get_driver(self):
    # WINDOWS
    # chromedriver_path = str("""C:/Users/Brian/Desktop/chromedriver.exe""")
    # user_data_path = "C:/Users/Brian/AppData/Local/Google/Chrome/User Data/Thread_Five"

    # MAC
    chromedriver_path = "/Users/brianphelps/Desktop/chromedriver"
    user_data_path = "/Users/brianphelps/Library/Application Support/Google/Chrome/Thread_Four"

    # OPTIONS
    options = webdriver.ChromeOptions()
    options.add_argument("no-sandbox")
    options.add_argument("disable-dev-shm-usage")
    options.add_argument("headless")
    options.add_argument(f"user-data-dir={user_data_path}")

    # START UP WEB DRIVER
    driver = webdriver.Chrome(chromedriver_path, chrome_options=options)
    return driver

  def _set_search(self, city,state):
    self.city=city
    self.state=state
    
    
  def fetch_data(self):
    link = "https://www.zillow.com/"
    
    driver = self._get_driver()
    
    @retry((ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
    def get_prop(link):
      driver.get(link)
    get_prop(link)

  
    input_element = driver.find_element_by_id("search-box-input")

    if city=='':
      query = state
    else:
      query = f"{city}, {state}"

    input_element.send_keys(query)

    driver.find_element_by_id("search-icon").click()
